This file needs requires external data!

(Run on workstation)

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.interpolate as si

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember


np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Input parameters

In [ ]:
#Interesting ensemble members:
#EC-Earth3-Veg PISM_DMI

In [ ]:
#Choose starting year
#year0 = 1951
year0 = 1871

#Choose ensemble member
#ism = 'ISSM_UCI'
#ism = 'GRIS_LSC'
#ism = 'IMAU_UU'
#ism = 'PISM_PIK'
#ism = 'PISM_DMI'
#ism = 'FETI_ULB'
ism = 'CISM_NCA'
#ism = 'MALI_DOE'
#ism = 'SICO_ILTS'
#esm = 'NorESM2-MM'
esm = 'EC-Earth3'
#esm = 'EC-Earth3-Veg'
#esm = 'ACCESS-CM2'
#esm = 'CanESM5'
#esm = 'CAS-ESM2-0'
#esm = 'CAMS-CSM1-0'
#esm = 'INM-CM4-8'
#esm = 'INM-CM5-0'
#esm = 'CMCC-ESM2'
#esm = 'MIROC6'
#esm = 'MRI-ESM2-0'
#esm = 'MPI-ESM1-2-LR'
#esm = 'GFDL-ESM4'
ssp = '585'

Computation

In [ ]:
#Read data
ad = AllData()
ad.fitctrl=True
ad.gather(update=False)

#print(ad.ism)

#Compute iteration of ensemble member
ens = EnsembleMember(ad,ism=ism,esm=esm,ssp=ssp,year0=year0)
#ens.niter=1
#ens.verbose=True
ens.usefanom = True
ens.compute()

print(ens.gamma)

Figure settings

In [ ]:
plt.style.use('../src/style_paper')

mpl.rc('figure.subplot',left=.15)

savename = f'../draftfigs/single_ensemble.png'

Create and save figure

In [ ]:

#Choose variables to plot
#varbs = [ens.TMP,ens.MLT,ens.IML,100*ens.SLR]
varbs = [ens.TMP,ens.IML,100*ens.SLR]

fig,ax = plt.subplots(len(varbs),5,sharex=True,sharey='row',figsize=(7,5))


for b,bas in enumerate(ens.basin.values):
    for v,var in enumerate(varbs):
        dax = ax[v,b]
        #Add reference lines
        if v==0:
            dax.axhline(ens.tref[b],0,1,c='.5',lw=.5)
        dax.axhline(0,0,1,c='k',lw=.5)

        #No feedback in grey
        dax.plot(ens.time,var[0,:,b],c='.5',lw=1,label='No feedback')
        
        #if v == 1:
        #    dax.text(.1,.8,f'{var[0,-1,b]:.1f}',transform=dax.transAxes)

        dax.plot(ens.time,var[1,:,b],c=ad.bcol[bas],lw=1,ls='--',label='Feedback b.r.')
        dax.plot(ens.time,var[2,:,b],c=ad.bcol[bas],lw=1,label='Feedback a.r.')
        #if v>1:
        #    dax.text(.1,.8,f"{100*((var[ens.nconv,-1,b]-var[ens.nconv,0,b])-(var[0,-1,b]-var[0,0,b]))/np.abs(var[0,-1,b]-var[0,0,b]):+.0f}%",transform=dax.transAxes,weight='bold',c=ad.bcol[bas])
        
    ax[0,b].set_title(bas)

ax[-1,2].set_xlabel('Time [years]')
ax[0,0].set_ylabel(r'Temperature [$\degree$C]')
#ax[1,0].set_ylabel('Basal melt \n anomaly \n [m/yr]')
ax[-2,0].set_ylabel('Anomalous \n ice mass loss \n [Gt/yr]')
ax[-1,0].set_ylabel('Cumulative \n sea level rise \n [cm]')
#ax[2,2].set_ylim([0,1])
ax[-1,-1].legend(fontsize=6.5)

#ax[0,0].set_ylim([-1,1])
#ax[1,1].set_ylim([-10000,10000])
#ax[2,2].set_ylim([-1,2])

plt.savefig(savename)
plt.show()